In [5]:
from overpass_api_request import queryOverpassAPI
from google_maps_api import GoogleMapsVisitorRetriever
import overpy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
api = overpy.Overpass()

In [3]:
res = queryOverpassAPI(api, [47.21716341184269, -1.54070794225733], radius = 20)
print(len(res))

33


In [16]:
# Usage example
retriever = GoogleMapsVisitorRetriever(os.environ.get("GOOGLE_MAPS_API"))
# Assume 'locations' is the list from the original data
visitor_frequencies = retriever.process_locations(res)[:10]
print(visitor_frequencies)

[('Galerie des Machines de l’île', {'rating': 4.6, 'total_ratings': 39050, 'address': 'Parc des Chantiers, Bd Léon Bureau, 44200 Nantes, France', 'longitude': -1.5640668, 'latitude': 47.2060317}), ("Musée d'histoire de Nantes", {'rating': 4.6, 'total_ratings': 1113, 'address': '4 Pl. Marc Elder, 44000 Nantes, France', 'longitude': -1.5497506, 'latitude': 47.2158291}), ('La Baigneuse au Miroir', {'rating': 4.4, 'total_ratings': 972, 'address': 'Pl. Marc Elder, 44000 Nantes, France', 'longitude': -1.5426011, 'latitude': 47.2198057}), ('Voyage en Éléphant', {'rating': 4.7, 'total_ratings': 643, 'address': '5 Bd Léon Bureau, 44200 Nantes, France', 'longitude': -1.5645343, 'latitude': 47.2061287}), ("L'Artichaut", {'rating': 4.8, 'total_ratings': 450, 'address': '8rue du marais, 44000 Nantes, France', 'longitude': -1.5564028, 'latitude': 47.2181544}), ('Pop Corn Labyrinthe Nantes', {'rating': 4.2, 'total_ratings': 262, 'address': '1 Chem. des Ruettes, 44470 Carquefou, France', 'longitude': 

In [24]:
locations = {visitor_frequencies[i][0]: visitor_frequencies[i][1] for i in range(len(visitor_frequencies))}

In [59]:
import math
from itertools import permutations
class RouteOptimizer:
    @staticmethod
    def haversine_distance(loc1, loc2):
        """Calculate geographical distance between two locations."""
        R = 6371  # Earth radius in kilometers
        lat1, lon1 = loc1['latitude'], loc1['longitude']
        lat2, lon2 = loc2['latitude'], loc2['longitude']
        
        dLat = math.radians(lat2 - lat1)
        dLon = math.radians(lon2 - lon1)
        a = (math.sin(dLat/2)**2 + 
             math.cos(math.radians(lat1)) * 
             math.cos(math.radians(lat2)) * 
             math.sin(dLon/2)**2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return R * c

    @classmethod
    def nearest_neighbor_route(cls, locations):
        """Optimize route using Nearest Neighbor algorithm."""
        unvisited = locations.copy()
        route = [unvisited.pop(0)]  # Start with first location
        
        while unvisited:
            current = route[-1][1]
            nearest = min(unvisited, key=lambda loc: cls.haversine_distance(current, loc[1]))
            route.append(nearest)
            unvisited.remove(nearest)
        
        return route

    @classmethod
    def calculate_total_route_distance(cls, route):
        """Calculate total route distance."""
        res = [cls.haversine_distance(route[i][1], route[i+1][1]) 
                   for i in range(len(route)-1)]
        res.append(0)
        return res

def optimize_nantes_route(locations):
    # Extract location data
    
    # Full permutation approach (for small number of locations)
    if len(locations) <= 10:
        best_route = min(
            permutations(locations), 
            key=lambda route: RouteOptimizer.calculate_total_route_distance(route)
        )
    else:
        # Nearest neighbor for larger sets
        best_route = RouteOptimizer.nearest_neighbor_route(locations)
    
    # Create detailed route information
    route_info = []
    distance_per_step = RouteOptimizer.calculate_total_route_distance(best_route)
    total_distance = sum(distance_per_step)
    for location in best_route:
        # Find original location details
        for entry, details in locations:
            if entry == location[0]:
                original_entry = entry
                original_details = details
        # original_entry = next(entry for entry, _ in locations if entry == location)
        # original_details = next(details for entry, details in locations if entry == location)
        
        route_info.append({
            'name': original_entry,
            'rating': original_details['rating'],
            'total_ratings': original_details['total_ratings'],
            'address': original_details['address'],
            'distance_till_next': distance_per_step.pop(0)
        })
    
    return {
        'optimized_route': route_info,
        'total_distance': total_distance
    }

result = optimize_nantes_route(visitor_frequencies)

print("Optimized Route:")
for location in result['optimized_route']:
    print(f"- {location['name']} (Rating: {location['rating']}, Address: {location['address']}) - {location['distance_till_next']:.2f} km")
print(f"\nTotal Distance: {result['total_distance']:.2f} km")

Optimized Route:
- Voyage en Éléphant (Rating: 4.7, Address: 5 Bd Léon Bureau, 44200 Nantes, France) - 0.04 km
- Galerie des Machines de l’île (Rating: 4.6, Address: Parc des Chantiers, Bd Léon Bureau, 44200 Nantes, France) - 0.25 km
- tv quiz (Rating: 5, Address: 1 Rue Léon Durocher, 44200 Nantes, France) - 1.10 km
- HAB Galerie (Rating: 4.5, Address: 21 Quai des Antilles, 44200 Nantes, France) - 2.33 km
- L'Artichaut (Rating: 4.8, Address: 8rue du marais, 44000 Nantes, France) - 0.57 km
- Musée d'histoire de Nantes (Rating: 4.6, Address: 4 Pl. Marc Elder, 44000 Nantes, France) - 0.70 km
- La Baigneuse au Miroir (Rating: 4.4, Address: Pl. Marc Elder, 44000 Nantes, France) - 9.50 km
- Pop Corn Labyrinthe Nantes (Rating: 4.2, Address: 1 Chem. des Ruettes, 44470 Carquefou, France) - 0.00 km

Total Distance: 14.48 km


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/QwQ-32B-Preview"

model = AutoModelForCausalLM.from_pretrained(
    model_name, use_fast=True, device_map="auto",

)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "How many r in strawberry."
messages = [
    {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


model-00002-of-00017.safetensors:  91%|#########1| 3.55G/3.90G [00:00<?, ?B/s]

model-00003-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00004-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00005-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]